In [8]:
import os
import json
import pandas as pd
import numpy as np
import gymnasium as gym
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecMonitor


## **User Define Class**

In [16]:
# 创建类
class Data_Class:

    # 内部嵌套类
    class Trajectory_Class:
        def __init__(self, traj_series):
            self.traj_list = traj_series
            self.length = len(traj_series)

        def get_single_traj(self, index):
            return json.loads(self.traj_list[index])

    def __init__(self, path):
        self.path = path

        # 原始数据
        self.trajs_prefer_list = []
        self.trajs_reject_list = []

        # 处理数据
        self.traj_prefer_list_list_tensor = []
        self.traj_reject_list_list_tensor = []

        # 启动函数
        self.load_data(path)
        self.convert(self.trajs_prefer_list, self.traj_prefer_list_list_tensor) # 数据转换
        self.convert(self.trajs_reject_list, self.traj_reject_list_list_tensor)
        print("Data loaded successfully")

    def load_data(self, path):
        data = pd.read_csv(path)

        self.trajs_prefer_list = Data_Class.Trajectory_Class(data['preferred'])   # list 数据
        self.trajs_reject_list = Data_Class.Trajectory_Class(data['rejected'])    # list 数据

    def convert(self,
                list_json: Trajectory_Class,
                traj_list_list_tensor):

        # 获取第0条轨迹的第0时刻样本来确定维度
        sample = list_json.get_single_traj(0)[0]
        state0 = np.array(sample['state'])
        action0 = np.array(sample['action'])

        # 获取 state action 维度
        self.dim_state = state0.size if state0.ndim == 0 else state0.shape[0]
        self.dim_action = action0.size if action0.ndim == 0 else action0.shape[0]

        # 数据批量转换 tensor
        for idx in range(list_json.length):
            traj = list_json.get_single_traj(idx)
            states, actions = [], []

            for time_i in traj:
                # 转换为 numpy，然后 torch tensor
                state_np = np.array(time_i['state'])
                action_np = np.array(time_i['action'])

                state_t = torch.from_numpy(state_np).float()
                action_t = torch.from_numpy(action_np).float()

                # 如果是一维标量，要展开成长度1向量
                state_t = state_t.view(-1)
                action_t = action_t.view(-1)

                states.append(state_t)
                actions.append(action_t)

            # 将列表堆成张量 [L_i, dim]
            states_tensor = torch.stack(states, dim=0)
            actions_tensor = torch.stack(actions, dim=0)

            # 将每条轨迹作为一个元组 (states, actions) 添加到列表中
            traj_list_list_tensor.append((states_tensor, actions_tensor))

# ——— 数据集与加载器 ———
class PreferenceDataset(Dataset):
    def __init__(self, pref, rej, gamma):
        assert len(pref) == len(rej)
        self.pref = pref
        self.rej = rej
        self.gamma = gamma

    def __len__(self):
        return len(self.pref)

    def __getitem__(self, idx):
        return (*self.pref[idx], *self.rej[idx])

# 创建 MLP 打分模型 #0000FF
class RewardMLP(nn.Module):
    def __init__(self, s_dim, a_dim, hidden_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(s_dim + a_dim, hidden_dim),  # 这里在构造神经网络
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, s, a):
        # s: [L_i, s_dim], a: [L_i, a_dim]
        x = torch.cat([s, a], dim=-1)
        return self.net(x).squeeze(-1)



In [17]:
# ———— 加载数据 ————
path = "trajectory_pairs.csv"
Data = Data_Class(path)

for i in range(10):
    print(i, len(Data.trajs_prefer_list.get_single_traj(i)), len(Data.trajs_reject_list.get_single_traj(i)))

# print(len(Data.trajs_prefer_list.get_single_traj(0)),  len(Data.trajs_reject_list.get_single_traj(0)))
# print(len(Data.trajs_prefer_list.get_single_traj(1)),  len(Data.trajs_reject_list.get_single_traj(1)))
# print(len(Data.trajs_prefer_list.get_single_traj(2)),  len(Data.trajs_reject_list.get_single_traj(2)))

# print("")
# print(len(Data.traj_prefer_list_list_tensor[0][0]), len(Data.traj_reject_list_list_tensor[0][0]))
# print(len(Data.traj_prefer_list_list_tensor[1][0]), len(Data.traj_reject_list_list_tensor[1][0]))

Data loaded successfully
0 200 200
1 200 144
2 200 180
3 200 194
4 200 96
5 200 125
6 200 108
7 200 106
8 200 166
9 173 97


## **Reward Model Training**

In [18]:

# ———— 超参数 ————
num_pairs = 200    # 偏好对数量
T = 50             # 期望最大轨迹长度（用于评估或其他需求）
s_dim = 4         # 状态维度 [角度, 角速度, 小车位置, 小车速度]
a_dim = 1         # 动作维度（推力）
gamma = 0.99      # 折扣因子
lr = 1e-4         # 学习率
batch_size = 16
num_epochs = 50

# ———— 加载数据 ————
path = "trajectory_pairs.csv"
Data = Data_Class(path)

# 自定义 collate_fn，保留变长序列
def variable_collate(batch):
    # batch: List of tuples (s_pref, a_pref, s_rej, a_rej)
    s_pf, a_pf, s_rj, a_rj = zip(*batch)
    return list(s_pf), list(a_pf), list(s_rj), list(a_rj)

# 准备训练
dataset = PreferenceDataset(
    Data.traj_prefer_list_list_tensor,
    Data.traj_reject_list_list_tensor,
    gamma
)
loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=variable_collate
)

reward_net = RewardMLP(Data.dim_state, Data.dim_action, hidden_dim=64) # 实例化 神经网络 MLP
optimizer  = optim.Adam(reward_net.parameters(), lr=lr)
loss_fn    = nn.BCEWithLogitsLoss()

# ——— 训练循环 ———
for epoch in range(1, num_epochs + 1):
    total_loss = 0.0
    for s_pref_list, a_pref_list, s_rej_list, a_rej_list in loader:
        R_pref_batch = []
        R_rej_batch  = []

        # 计算 prefer 轨迹的回报
        for s_pf, a_pf in zip(s_pref_list, a_pref_list):
            r_pf = reward_net(s_pf, a_pf)           # [L_i]
            discounts = torch.tensor([gamma**t for t in range(r_pf.size(0))], device=r_pf.device)
            R_pref_batch.append((r_pf * discounts).sum())

        # 计算 reject 轨迹的回报
        for s_rj, a_rj in zip(s_rej_list, a_rej_list):
            r_rj = reward_net(s_rj, a_rj)          # [L_j]
            discounts = torch.tensor([gamma**t for t in range(r_rj.size(0))], device=r_rj.device)
            R_rej_batch.append((r_rj * discounts).sum())

        R_pref = torch.stack(R_pref_batch)
        R_rej = torch.stack(R_rej_batch)

        logits = R_pref - R_rej
        targets = torch.ones_like(logits)        # pref 应得更高分
        loss = loss_fn(logits, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(R_pref_batch)

    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch}/{num_epochs} — Avg Loss: {avg_loss:.4f}")

print("🎉 合成数据上的奖励模型训练完成！")

# ——— 保存模型 ———
torch.save(reward_net.state_dict(), 'reward_net.pth')
print("模型已保存到 reward_net.pth")


Data loaded successfully
Epoch 1/50 — Avg Loss: 1.8811
Epoch 2/50 — Avg Loss: 1.5743
Epoch 3/50 — Avg Loss: 1.3052
Epoch 4/50 — Avg Loss: 1.0893
Epoch 5/50 — Avg Loss: 0.9303
Epoch 6/50 — Avg Loss: 0.7888
Epoch 7/50 — Avg Loss: 0.6872
Epoch 8/50 — Avg Loss: 0.5927
Epoch 9/50 — Avg Loss: 0.5236
Epoch 10/50 — Avg Loss: 0.4684
Epoch 11/50 — Avg Loss: 0.4259
Epoch 12/50 — Avg Loss: 0.3925
Epoch 13/50 — Avg Loss: 0.3677
Epoch 14/50 — Avg Loss: 0.3490
Epoch 15/50 — Avg Loss: 0.3347
Epoch 16/50 — Avg Loss: 0.3233
Epoch 17/50 — Avg Loss: 0.3135
Epoch 18/50 — Avg Loss: 0.3075
Epoch 19/50 — Avg Loss: 0.3004
Epoch 20/50 — Avg Loss: 0.2953
Epoch 21/50 — Avg Loss: 0.2907
Epoch 22/50 — Avg Loss: 0.2864
Epoch 23/50 — Avg Loss: 0.2825
Epoch 24/50 — Avg Loss: 0.2795
Epoch 25/50 — Avg Loss: 0.2760
Epoch 26/50 — Avg Loss: 0.2737
Epoch 27/50 — Avg Loss: 0.2711
Epoch 28/50 — Avg Loss: 0.2686
Epoch 29/50 — Avg Loss: 0.2666
Epoch 30/50 — Avg Loss: 0.2649
Epoch 31/50 — Avg Loss: 0.2622
Epoch 32/50 — Avg Loss:

## **Reward Model Testing**

In [19]:
# ——— 加载模型示例 ———
reward_net_loaded = RewardMLP(Data.dim_state, Data.dim_action, hidden_dim=64)
reward_net_loaded.load_state_dict(torch.load('reward_net.pth', weights_only=True))

# 切到推理模式，并关闭梯度
reward_net_loaded.eval()
print("加载并准备好进行推理")

# 加载数据
for i in range(10):
    traj_prefer_json = Data.trajs_prefer_list.get_single_traj(i)
    traj_reject_json = Data.trajs_reject_list.get_single_traj(i)

    # 把它转成张量
    states_prefer  = torch.stack([torch.from_numpy(np.array(step['state'])).float().view(-1)
                                  for step in traj_prefer_json], dim=0)  # [L, s_dim]
    actions_prefer = torch.stack([torch.from_numpy(np.array(step['action'])).float().view(-1)
                                    for step in traj_prefer_json], dim=0)  # [L, a_dim]
    states_reject  = torch.stack([torch.from_numpy(np.array(step['state'])).float().view(-1)
                                    for step in traj_reject_json], dim=0)  # [L, s_dim]
    actions_reject = torch.stack([torch.from_numpy(np.array(step['action'])).float().view(-1)
                                    for step in traj_reject_json], dim=0)  # [L, a_dim]

    # 计算 prefer 轨迹的回报
    with torch.no_grad():
        r_pref = reward_net_loaded(states_prefer, actions_prefer)           # [L_i]

    # 计算总回报
    discounts = torch.tensor([gamma**t for t in range(r_pref.size(0))])
    total_return_prefer = (r_pref * discounts).sum()

    # 计算 reject 轨迹的回报
    with torch.no_grad():
        r_rj = reward_net_loaded(states_reject, actions_reject)          # [L_j]

    # 计算总回报
    discounts = torch.tensor([gamma**t for t in range(r_rj.size(0))])
    total_return_reject = (r_rj * discounts).sum()

    print(i, total_return_prefer, total_return_reject)



加载并准备好进行推理
0 tensor(10.9267) tensor(9.9196)
1 tensor(9.7878) tensor(7.5548)
2 tensor(11.4603) tensor(9.7593)
3 tensor(10.9521) tensor(9.6708)
4 tensor(11.2456) tensor(5.3581)
5 tensor(11.0701) tensor(6.5134)
6 tensor(10.5024) tensor(5.3424)
7 tensor(10.8763) tensor(5.1974)
8 tensor(10.2842) tensor(9.6710)
9 tensor(10.4395) tensor(6.1724)


## **RLHF 模型训练**

In [7]:
# --------------------------------------------------------------------------------------------------
# 2. 定义一个 Wrapper，在 step 里用你的 MLP 计算 reward
# --------------------------------------------------------------------------------------------------
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env, reward_model_path, device="cpu"):
        super().__init__(env)
        # 创建并加载你的 MLP 奖励模型

        # state 维度
        self.dim_state = env.observation_space.shape[0]

        # action 维度
        try:                  self.dim_action = env.action_space.shape[0]
        except IndexError:    self.dim_action = 1

        self.device = device

        # Ensure the input dimensions match the checkpoint
        checkpoint = torch.load(reward_model_path, map_location=device, weights_only=False)
        input_dim = checkpoint['net.0.weight'].size(1)  # Extract input size from checkpoint
        self.reward_model = RewardMLP(input_dim - self.dim_action, self.dim_action).to(device)
        self.reward_model.load_state_dict(checkpoint)
        self.reward_model.load_state_dict(torch.load(reward_model_path, map_location=device, weights_only=False))
        self.reward_model.eval()
    
    def step(self, action):
        # 执行原 env，不用原 reward
        obs, _, terminated, truncated, info = self.env.step(action)   
        
        # 转成 batch 形式再丢给网络
        state_tensor = torch.tensor(obs, dtype=torch.float32, device=self.device).unsqueeze(0)



        if isinstance(self.env.action_space, gym.spaces.Discrete):
            action_tensor = torch.tensor([action], dtype=torch.long, device=self.device)
        else:
            action_tensor = torch.tensor(action, dtype=torch.float32, device=self.device).unsqueeze(0)

        # 修改 action_tensor 形状
        if action_tensor.ndim == 1:
            action_tensor = action_tensor.view(1, -1)

        # 计算奖励
        with torch.no_grad():
            reward_tensor = self.reward_model(state_tensor, 
                                              action_tensor)
        reward = reward_tensor.item()
        return obs, reward, terminated, truncated, info
    
def reset(self, **kwargs):
    # 接受 seed, options, return_info 等任意参数
    return self.env.reset(**kwargs)


# --------------------------------------------------------------------------------------------------
# 3. 构造 vectorized 环境，并应用自定义 Wrapper
# --------------------------------------------------------------------------------------------------

# Log 地址
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_path = os.path.join("Training", current_time)
os.makedirs(log_path, exist_ok=True)

# Reward Model 地址
MODEL_PATH = "reward_net.pth"

# 如果你用 SB3 的 make_vec_env，可以传 wrapper_class
vec_env = make_vec_env(
    env_id="CartPole-v1",
    n_envs=8,
    wrapper_class=lambda env: CustomRewardWrapper(env, MODEL_PATH, device="cpu"),
    monitor_dir=log_path
)
# 再加一个 Monitor（记录 episode reward 到文件）
vec_env = VecMonitor(vec_env, log_path)


model = PPO(
    policy="MlpPolicy",
    env=vec_env,
    n_steps=256,
    device="cpu",
    verbose=1,
    tensorboard_log=log_path
)

model.learn( 
    total_timesteps=50000,
    # callback=[eval_callback, save_callback]
)

model.save(os.path.join(log_path, "model_full_training"))

NameError: name 'gym' is not defined

## **RLHF 模型测试部分**

In [ ]:
import os
import sys

cur_dir = os.getcwd()
pkg_dir = os.path.dirname(cur_dir)

if pkg_dir not in sys.path:
    sys.path.append(pkg_dir)

from Project import tools


# 评估模型
log_path = "Training\\2025-05-08_19-59-36"
PPO_Model_Path = os.path.join(log_path, "model_full_training")
tools.test_model("PPO", PPO_Model_Path, n_episodes=2, render = True, record=True)

Training\2025-05-08_19-59-36\model_full_training
Episode: 1 Score: [500.]
Episode: 2 Score: [500.]
